# Letter Prediction CNN Deployment

This is the deployment of my neural network that is used to predict handwritten letters

### Import packages

In [1]:
import cv2
import numpy as np 
import sqlite3
import sys
import os 
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image

## Generate our SQL db connection 
We are using sqlite for this implementation

In [2]:

conn = sqlite3.connect('Predictions_letters.db')
  
sql_create_pred_table = """ CREATE TABLE IF NOT EXISTS letter_pred(
                                        id integer PRIMARY KEY,
                                        prediction text NOT NULL,
                                        image BLOB NOT NULL
                                        
                                    ); """

c = conn.cursor()
c.execute(sql_create_pred_table)

## Deployment of Model using Tkinter, PIL, cv2

In [9]:

#load the trained model to classify sign
from keras.models import load_model
model = load_model('model_handwriting_final_model.h5')
#dictionary to label all traffic signs class.
classes ={0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',
             13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X', 24:'Y',25:'Z'}
#initialise GUI
top=tk.Tk()
top.geometry('800x600')
top.title('Letter Classification')
top.configure(background='#CDCDCD')
label=Label(top,background='#CDCDCD', font=('arial',15,'bold'))
sign_image = Label(top)

def classify(file_path):
 
    global label_packed
    img = cv2.imread(file_path)
    img_copy = img.copy()

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (400,440))
    img_copy = cv2.GaussianBlur(img_copy, (7,7), 0)
    image = Image.open(file_path)
    img_copy = cv2.GaussianBlur(img_copy, (7,7), 0)
    img_gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
    _, img_thresh = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV)

    img_final = cv2.resize(img_thresh, (28,28))
    img_final =np.reshape(img_final, (1,28,28,1))
    pred = model.predict_classes([img_final])[0]
    sign = classes[pred]
    label.configure(foreground='#011638', text="Letter Prediction: " + str(sign))
    
    try:
        
        cursor = conn.cursor()
        print("Connected to SQLite")
        sqlite_insert_blob_query = """ INSERT INTO letter_pred
            (id, prediction, image) VALUES (?, ?, ?)"""
        cursor.execute("select count(*) from letter_pred")
        values = cursor.fetchone()
        index = (int(values[0]+1))
        # remember order is id, prediction, image
        data_tuple = (index, sign, file_path)
        cursor.execute(sqlite_insert_blob_query, data_tuple)
        conn.commit()
        print("Image and file inserted successfully as a BLOB into a table")
        cursor.close()
    except:
        print("error")
def restart_full():
        os.execl(sys.executable, sys.executable, *sys.argv)
def show_classify_button(file_path):
    classify_b=Button(top,text="Classify Image",command=lambda: classify(file_path),padx=10,pady=5)
    classify_b.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
    classify_b.place(relx=0.79,rely=0.46)
def upload_image():
    try:
        file_path=filedialog.askopenfilename()
        uploaded=Image.open(file_path)
        uploaded.thumbnail(((top.winfo_width()/2.25),(top.winfo_height()/2.25)))
        im=ImageTk.PhotoImage(uploaded)
        sign_image.configure(image=im)
        sign_image.image=im
        label.configure(text='')
        show_classify_button(file_path)
    except:
        pass
restart=Button(top,text="Restart",command=restart_full,padx=10,pady=5)
restart.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
restart.pack(side=BOTTOM,pady=50)
upload=Button(top,text="Upload an image",command=upload_image,padx=10,pady=5)
upload.configure(background='#364156', foreground='white',font=('arial',10,'bold'))
upload.pack(side=BOTTOM,pady=50)


sign_image.pack(side=BOTTOM,expand=True)
label.pack(side=BOTTOM,expand=True)
heading = Label(top, text="Upload a letter image to be predicted",pady=20, font=('arial',20,'bold'))
heading.configure(background='#CDCDCD',foreground='#364156')
heading.pack()
top.mainloop()



Connected to SQLite
Image and file inserted successfully as a BLOB into a table


## Some example functions to query the db

In [11]:
import random
c = conn.cursor()
#c.execute('DELETE FROM letter_pred')
def open_image_with_index(ind):
   
    c.execute("SELECT image from letter_pred WHERE id =?",str(ind))
    x = c.fetchone()

    im = Image.open(x[0])
    im.show()
    
#open_image_with_index(1)
def open_first_prediction_of_letter(letter):
    
    c.execute('''SELECT image from letter_pred WHERE prediction =  ?''',letter)
    x = c.fetchall()
    im = Image.open(x[0][0]) 
    im.show()
#open_first_prediction_of_letter("B")
def select_random_image():
    c.execute("select count(*) from letter_pred")
    values = c.fetchone()
    db_len= (int(values[0]))
    if db_len >0:
        random_number = random.randint(1,db_len )
        open_image_with_index(random_number)
    else:
        print("DB is empty")

select_random_image()

DB is empty
